In [198]:
import os;
import json;
import builtins
import pandas as pd;

dir = '/home/chesetti/Repos/learned_merge_cleanup/sponge'
testcase = 'fb_join-onelevel_1'
test_dir = os.path.join(dir, testcase)
results_dir = os.path.join(test_dir, 'outputs', 'results')
csv_dir = os.path.join(test_dir, 'csv')
os.makedirs(csv_dir, exist_ok=True)

runs = [os.path.join(results_dir, run) for run in os.listdir(results_dir)]
print(runs)

FileNotFoundError: [Errno 2] No such file or directory: '/home/chesetti/Repos/learned_merge_cleanup/sponge/fb_merge-onelevel_1/outputs/results'

In [ ]:
# Walk all the json files and put them in a dataframe

test_results = []
for run in runs:
    for test_result_file in os.listdir(run):
        json_file = builtins.open(os.path.join(run, test_result_file))
        test_result = json.load(json_file)
        test_result['run'] = run
        test_results.append(test_result)
        json_file.close()
test_dataframe = pd.json_normalize(test_results)
print(test_dataframe.columns)

        

Index(['command', 'run', 'result.checksum', 'result.duration_ns',
       'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_index_build_duration_ns', 'result.inner_index_size',
       'result.outer_disk_fetch', 'spec.algo', 'spec.algo_name',
       'spec.common_key', 'spec.index.search', 'spec.index.type',
       'spec.inner_table', 'spec.key_size', 'spec.key_type',
       'spec.load_sstable_in_mem', 'spec.name', 'spec.num_threads',
       'spec.outer_table', 'spec.result_path', 'spec.value_size',
       'spec.write_result_to_disk', 'spec.index.leaf_size_in_pages'],
      dtype='object')


# Reports

### Overall Duration

In [ ]:
overall_duration = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.duration_ns', aggfunc='median')
overall_duration.to_csv(os.path.join(csv_dir, 'duration_sec.csv'))
print(overall_duration)

spec.algo_name       BTree    PGM1024     PGM256      PGM64  standard_merge
spec.common_key                                                            
1                606160089  366958773  373037464  377308888       285113650
2                317003678  212197200  213604378  209402956       247252022
3                252606097  159362202  156869322  156202667       208862321
4                178062653  123901170  126693906  139224538       167555758
5                152279125  109620721  116857813  125542114       179591942
6                148263693  112560342  107039107  102140839       174632651
7                123130904   94173019   99481311  103293318       170801862
8                126574979   93564080   94215218   88969271       147882179
9                105425032   89020925   87554369   84272262       145322552
10                99510093   84620707   85804130   82344143       141392439


In [ ]:
inner_index_size = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_index_size', aggfunc='median')
data = {}
data['indexes'] = []
data['memory_bytes'] = []
for index in inner_index_size:
    data['indexes'].append(index)
    data['memory_bytes'].append(inner_index_size[index].mean())
inner_index_size = pd.DataFrame(data=data)
inner_index_size.to_csv(os.path.join(csv_dir, 'inner_index_size.csv'))
print(inner_index_size)

          indexes  memory_bytes
0           BTree       20592.0
1         PGM1024         152.0
2          PGM256        1032.0
3           PGM64       13744.0
4  standard_merge      161664.0


In [ ]:
inner_index_build_duration = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_index_build_duration_ns', aggfunc='median')
data = {}
data['indexes'] = []
data['build_duration'] = []
for index in inner_index_build_duration:
    data['indexes'].append(index)
    data['build_duration'].append(inner_index_build_duration[index].mean())
inner_index_build_duration = pd.DataFrame(data=data)
inner_index_build_duration.to_csv(os.path.join(csv_dir, 'inner_index_build_duration_ns.csv'))

In [ ]:
inner_index_fetch = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_disk_fetch', aggfunc='median')
print(inner_index_fetch)

spec.algo_name   BTree  PGM1024  PGM256  PGM64  standard_merge
spec.common_key                                               
1                 7774     7774    7774   7774            7774
2                 7775     7775    7775   7775            7774
3                 7775     7775    7775   7775            7774
4                 7774     7774    7774   7774            7774
5                 7774     7774    7774   7774            7774
6                 7774     7774    7774   7774            7774
7                 7774     7774    7774   7774            7774
8                 7774     7774    7774   7774            7774
9                 7774     7774    7774   7774            7774
10                7774     7774    7774   7774            7774


In [ ]:
result_checksum = (test_dataframe[['spec.common_key', 'spec.algo_name', 'result.checksum']].sort_values(by=['spec.common_key', 'spec.algo_name'])) #.loc[test_dataframe['spec.common_key']=='10'])
for common_key in sorted(test_dataframe['spec.common_key'].unique()):
    checksums = result_checksum.loc[result_checksum['spec.common_key'] == common_key]
    unique_checksums = checksums['result.checksum'].unique()
    if (len(unique_checksums) == 1):
        print(f"common_key: {common_key} checksum: {unique_checksums} OK")
    else:
        print(f"common_key: {common_key} checksums don't match")
print(result_checksum.loc[result_checksum['spec.common_key'] == 40])
print(result_checksum.loc[result_checksum['spec.common_key'] == 40]['result.checksum'].unique())

common_key: 1 checksum: ['FB1325CD1EF6A662FDB62437963A8DD5'] OK
common_key: 2 checksum: ['B546A407990E8FCAD0E4BAC58C5665D8'] OK
common_key: 3 checksum: ['4FC3A19486DB5C190F9AD70876BDEDF8'] OK
common_key: 4 checksum: ['851AEB57B15C646D1558BA702BDF7F63'] OK
common_key: 5 checksum: ['055B1A29A1A05237AB91998CAB7A7CA6'] OK
common_key: 6 checksum: ['DA06D0BB6A18417D754255F6D8A124AD'] OK
common_key: 7 checksum: ['C1B3AD5394742782DCFA4C9CF924908D'] OK
common_key: 8 checksum: ['671B084C9E572A8BEB928111959D1A98'] OK
common_key: 9 checksum: ['9AC46D4F7E8FEDD804D7E6495D6C0155'] OK
common_key: 10 checksum: ['3985DF0E03E87AFA419795428B2DFCBB'] OK
Empty DataFrame
Columns: [spec.common_key, spec.algo_name, result.checksum]
Index: []
[]
